In [1]:
import spacy
import pandas as pd

In [2]:
# Join the reports
reports_de = pd.read_csv('reports_de.csv')
reports_en = pd.read_csv('reports_en.csv')
reports_joined = reports_de.append(reports_en)
reports_joined.to_csv('reports_joined.csv')

In [3]:
#reports = pd.read_csv('reports.csv')
reports = pd.read_csv('reports_joined.csv')
reports['text'] = reports['text'].str.replace(r'\s+',' ', regex=True)
reports.head()

,Unnamed: 0,Unnamed: 0.1,filepath,number_of_pages,lang,text
0,0,0,./reports/scraped/full/mtu.pdf,226,deu,Aero Engines MTU SS Geschäftsbericht MTU AERO ...
1,1,1,./reports/scraped/full/henkel.pdf,219,deu,Henkel Nachhaltigkeitsbericht 2020 02 Vorwort...
2,2,3,./reports/scraped/full/dpdhl.pdf,2,deu,NACHHALTIGKEITSFAHRPLAN ZAHLEN & FAKTEN NACHH...
3,3,5,./reports/scraped/full/daimler.pdf,205,deu,DAIMLER Nachhaltigkeitsbericht 2019 SIE Vorwor...
4,4,14,./reports/scraped/full/deutsche-boerse.pdf,7,deu,DEUTSCHE BÖRSE it GROUP Klimastrategie der Gr...


In [11]:
# from langdetect import detect
# reports['lang'] = reports['text'].apply(lambda text: detect(text[:500]))
# reports.head()

,Unnamed: 0,filepath,number_of_pages,text,text_processed,lang
0,0,./reports/scraped/full/mtu.pdf,226,Aero Engines MTU Lo Geschdftsbericht MTU AERO ...,Aero Engines MTU Lo Geschdftsbericht MTU AERO ...,de
1,1,./reports/scraped/full/henkel.pdf,219,Henkel Nachhaltigkeitsbericht 2020 02 Vorwort...,Henkel Nachhaltigkeitsbericht 2020 02 Vorwor...,de
2,2,./reports/scraped/full/allianz.pdf,102,|COLLABORATING FOR A SUSTAINABLE FUTURE es | a...,"|collaborate SUSTAINABLE future es | , | ALLIA...",en
3,3,./reports/scraped/full/dpdhl.pdf,2,NACHHALTIGKEITSFAHRPLAN ZAHLEN & FAKTEN NACHH...,NACHHALTIGKEITSFAHRPLAN ZAHLEN & FAKTEN nach...,de


In [3]:
# nlp = spacy.load('en_core_web_sm')

In [12]:
# doc = nlp(reports['text'].values[0])

nlp_en = spacy.load('en_core_web_sm')
nlp_en.max_length = 1300000
nlp_de = spacy.load('de_core_news_sm')
nlp_de.max_length = 1300000

# New stop words list 
customize_stop_words = [
    'attach'
]

def lemmatize(nlp, text):
    # Mark them as stop words
    for w in customize_stop_words:
        nlp.vocab[w].is_stop = True
    return " ".join(token.lemma_ for token in nlp(text) 
                                if not token.is_stop)

# index, row
for row in reports.iterrows():
    row = row[1]
    if row.lang == 'eng':
        # doc = nlp_en(row['text'])
        row['lemma'] = lemmatize(nlp_en, row['text'])
    if row.lang == 'deu':
        row['lemma'] = lemmatize(nlp_de, row['text'])


# Mark them as stop words
# for w in customize_stop_words:
#     nlp.vocab[w].is_stop = True


# # Test data
# # df = pd.DataFrame( {'Sumcription': ["attach poster on the wall because it is cool",
# #                                    "eating and sleeping"]})

# # Convert each row into spacy document and return the lemma of the tokens in 
# # the document if it is not a sotp word. Finally join the lemmas into as a string
# reports['text_processed'] = reports.text.apply(lambda text: 
#                                           " ".join(token.lemma_ for token in nlp(text) 
#                                                    if not token.is_stop))

print (reports.head())

   Unnamed: 0  Unnamed: 0.1                                    filepath  \
0           0             0              ./reports/scraped/full/mtu.pdf   
1           1             1           ./reports/scraped/full/henkel.pdf   
2           2             3            ./reports/scraped/full/dpdhl.pdf   
3           3             5          ./reports/scraped/full/daimler.pdf   
4           4            14  ./reports/scraped/full/deutsche-boerse.pdf   

   number_of_pages lang                                               text  
0              226  deu  Aero Engines MTU SS Geschäftsbericht MTU AERO ...  
1              219  deu   Henkel Nachhaltigkeitsbericht 2020 02 Vorwort...  
2                2  deu   NACHHALTIGKEITSFAHRPLAN ZAHLEN & FAKTEN NACHH...  
3              205  deu  DAIMLER Nachhaltigkeitsbericht 2019 SIE Vorwor...  
4                7  deu   DEUTSCHE BÖRSE it GROUP Klimastrategie der Gr...  


In [15]:
print(f"Token \t\tLemma \t\tStopword".format('Token', 'Lemma', 'Stopword'))
print("-"*40)
for token in doc:
    print(f"{str(token)}\t\t{token.lemma_}\t\t{token.is_stop}")

9		False
Facts		fact		False
&		&		False
figures		figure		False
publication		publication		False
tcontrioutors		tcontrioutor		False
Published		publish		False
on		on		True
April		April		False
14		14		False
,		,		False
2020		2020		False
by		by		True
Merck		Merck		False
,		,		False
Group		Group		False
Corporate		Corporate		False
Responsibility		Responsibility		False
Frankfurter		Frankfurter		False
Strasse		Strasse		False
250		250		False
,		,		False
64293		64293		False
Darmstadt		Darmstadt		False
,		,		False
Germany		Germany		False
Tel		Tel		False
.		.		False
:		:		False
+49		+49		False
(		(		False
0		0		False
)		)		False
6151		6151		False
-		-		False
72		72		False
0		0		False
Fax		fax		False
:		:		False
+49		+49		False
(		(		False
0		0		False
)		)		False
6151		6151		False
-		-		False
72		72		False
5577		5577		False
Website		website		False
:		:		False
www.merckgroup.com		www.merckgroup.com		False
Contact		Contact		False
for		for		True
questions		question		False
regarding		regard		True
this		

In [5]:
# https://stackoverflow.com/questions/55817040/removing-stop-words-using-spacy#55818253
import spacy
import pandas as pd

# Load spacy model
nlp = spacy.load('en_core_web_sm')

# New stop words list 
customize_stop_words = [
    'attach'
]

# Mark them as stop words
for w in customize_stop_words:
    nlp.vocab[w].is_stop = True


# Test data
# df = pd.DataFrame( {'Sumcription': ["attach poster on the wall because it is cool",
#                                    "eating and sleeping"]})

# Convert each row into spacy document and return the lemma of the tokens in 
# the document if it is not a sotp word. Finally join the lemmas into as a string
reports['text_processed'] = reports.text.apply(lambda text: 
                                          " ".join(token.lemma_ for token in nlp(text) 
                                                   if not token.is_stop))

print (reports.head())


   Unnamed: 0                            filepath  number_of_pages  \
0           0      ./reports/scraped/full/mtu.pdf              226   
1           1   ./reports/scraped/full/henkel.pdf              219   
2           2  ./reports/scraped/full/allianz.pdf              102   
3           3    ./reports/scraped/full/dpdhl.pdf                2   

                                                text  \
0  Aero Engines MTU Lo Geschdftsbericht MTU AERO ...   
1   Henkel Nachhaltigkeitsbericht 2020 02 Vorwort...   
2  |COLLABORATING FOR A SUSTAINABLE FUTURE es | a...   
3   NACHHALTIGKEITSFAHRPLAN ZAHLEN & FAKTEN NACHH...   

                                      text_processed  
0  Aero Engines MTU Lo Geschdftsbericht MTU AERO ...  
1    Henkel Nachhaltigkeitsbericht 2020 02 Vorwor...  
2  |collaborate SUSTAINABLE future es | , | ALLIA...  
3    NACHHALTIGKEITSFAHRPLAN ZAHLEN & FAKTEN nach...  


In [13]:
reports.to_csv('processed.csv')
processed = pd.read_csv('processed.csv')
processed['text'].values[0]

Verbindlichkeiten [T103) 169 Investitionen in immaterielle Vermögenswerte, Sachanlagen, Angaben zu Finanzinstrumenten Buchwerte, Wertansätze und erworbene Programmwerte und Entwicklung [T133] 198 beizulegende Zeitwerte zum 31.12.2020 [11041 170 Langfristige Vermögenswerte [1134] 198 Angaben zu Finanzinstrumenten Buchwerte, Wertansätze und Gewinn- und Verlustrechnung der MTU Aero Engines AG [T135] 200 beizulegende Zeitwerte zum 31.12.2019 [105] 171 Hierarchische Einstufung der zum beizulegenden Zeitwert bewerteten finanziellen Vermögenswerte und Verbindlichkeiten für das Geschäftsjahr 2020 [T106) 172 223 Geschäftsbericht 2020 Weitere Informationen Finanzkalender 21. April 2021 Hauptversammlung 30. April 2021 Quartalsmitteilung zum 31. März 2021 Telefonkonferenzen mit Journalisten, Analysten und Investoren 30. Juli 2021 Zwischenbericht zum 30. Juni 2021 Telefonkonferenzen mit Journalisten, Analysten und Investoren 29. Oktober 2021 Quartalsmitteilung zum 30. September 2021 Telefonkonferen

In [7]:
processed['text_processed'].values[0]

swerte , Sachanlagen , Angaben zu Finanzinstrumenten Buchwerte , Wertansatze und erworbene Programmwerte und Entwicklung ( T133 ] 198 beizulegende Zeitwerte zum 31.12.2020 ( T1041 170 Langfristige Vermégenswerte T1341 ] 198 Angaben zu Finanzinstrumenten Buchwerte , Wertansatze und Gewinn- und Verlustrechnung der MTU Aero Engines AG ( T135 ] 200 beizulegende Zeitwerte zum 31.12.2019 ( 1105 ) 171 Hierarchische Einstufung der zum beizulegenden Zeitwert bewerteten finanziellen Vermégenswerte und Verbindlichkeiten fiir das Geschaftsjahr 2020 ( t106 ] 172 223 Geschaftsbericht 2020 Weitere Informationen Finanzkalender 21 . April 2021 Hauptversammlung 30 . April 2021 Quartalsmitteilung zum 31 . Marz 2021 Telefonkonferenzen mit Journalisten , Analysten und Investoren 30 . Juli 2021 Zwischenbericht zum 30 . Juni 2021 Telefonkonferenzen mit Journalisten , Analysten und Investoren 29 . oktober 2021 Quartalsmitteilung zum 30 . September 2021 Telefonkonferenzen mit Journalisten , Analysten und Inves

In [14]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
#% matplotlib inline

# Create and generate a word cloud image:
wordcloud = WordCloud().generate(processed['text_processed'].values[0])

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

KeyError: 'text_processed'